In [10]:
import os, glob, warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = "./115"

SEED = 42
np.random.seed(SEED)



In [11]:
paths = sorted(glob.glob(os.path.join(DATA_DIR, "**", "*.*"), recursive=True))
print(f"Nájdených súborov: {len(paths)}")

for p in paths[:10]:
    print(p)

Nájdených súborov: 3
./115/observation.csv
./115/patient.csv
./115/station.csv


In [8]:
# Pokus o načítanie tabuliek do dictu `loaded`
loaded = {}

for p in paths:
    ext = p.split(".")[-1].lower()
    try:
        if ext == "csv":
            df = pd.read_csv(p)
            loaded[os.path.basename(p)] = df
        elif ext in ("xlsx", "xls"):
            # ak máš Excel s viacerými listami
            xls = pd.read_excel(p, sheet_name=None)
            for sheet, dfx in xls.items():
                loaded[f"{os.path.basename(p)}::{sheet}"] = dfx
        else:
            continue
    except Exception as e:
        print(f"Preskočené {p}: {e}")

print(f"Načítaných tabuliek: {len(loaded)}")
list(loaded.keys())

Preskočené ./115/patient.csv: Error tokenizing data. C error: Expected 1 fields in line 6, saw 3

Preskočené ./115/station.csv: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2

Načítaných tabuliek: 1


['observation.csv']